In [1]:
# !pip install git+https://github.com/jbusecke/cmip6_preprocessing.git@refs/pull/187/head

## Outline

- Access to CMIP6 data
    - How do i get to the data?
    - How do I select certain variables/experiments/models?
        - Handy CSV from Naomi
- Why would I need a preprocessing package?
    - Examples of messed up conventions
    - combined preprocessing
    
- Combining datasets for analysis
    - Matching metrics
    - Combining datasets

- Masking out ocean basins

- Removing trends from the data

In this tutorial you will learn how to work with CMIP6 data in the cloud.
This includes:
- Finding and selecting data
- Loading the data
- Combining different datasets
- Masking out ocean basins
- Removing model drift
    
All of these assume a basic knowledge of python and xarray.

## What is CMIP6?


...

## Where do I find the data?

The official CMIP6 archive is maintained by [ESGF](https://esgf-node.llnl.gov/projects/cmip6/), but downloadind data from there can be a time and storage consuming task. The pangeo Project has mirrored a substantial part of the archive to the [cloud](https://pangeo-data.github.io/pangeo-cmip6-cloud/) (not all of them though, so if you are missing something always check on the ESGF site first).

So now that you know where the data is located (we are going to use the data on Google Cloud, since this is where our computations will happen). 

All the CMIP6 variables follow a stric vocabulary, so you might want to check out this handy [spreadsheet by Naomi Henderson](https://docs.google.com/spreadsheets/d/1UUtoz6Ofyjlpx5LdqhKcwHFz2SGoTQV2_yekHyMfL9Y/edit#gid=1221485271)

Ok then lets get started...

In [2]:
import matplotlib.pyplot as plt
import intake

In [3]:
from cmip6_preprocessing.utils import google_cmip_col

# Initialize the Pangeo CMIP6 cloud collection
col = google_cmip_col()

The collection is based on a pandas dataframe:


In [4]:
col.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,hfss,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,uas,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,6hrPlev,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,clt,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
...,...,...,...,...,...,...,...,...,...,...,...
511171,CMIP,CMCC,CMCC-CM2-HR4,abrupt-4xCO2,r1i1p1f1,Amon,clt,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-HR4/abrupt...,NaN,20210624
511172,CMIP,CMCC,CMCC-CM2-HR4,abrupt-4xCO2,r1i1p1f1,Amon,ts,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-HR4/abrupt...,NaN,20210624
511173,CMIP,CMCC,CMCC-CM2-HR4,abrupt-4xCO2,r1i1p1f1,day,hfls,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-HR4/abrupt...,NaN,20210624
511174,ScenarioMIP,AS-RCEC,TaiESM1,ssp585,r1i1p1f1,day,tasmin,gn,gs://cmip6/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/s...,NaN,20210721


The column names represent another set of 'vocabulary' that is useful to learn. Especially the values of `source_id` (the model), `experiment_id` the experiment that was run, and `table_id`(time frequency of output) will apear again and again.
We could just find the dataset we want to analyze and [manually load it with xarray](https://pangeo-data.github.io/pangeo-cmip6-cloud/accessing_data.html#opening-a-single-zarr-data-store), but that is cumbersome.

Instead we are using the abilities of the `col` object, powered by [intake-esm](https://intake-esm.readthedocs.io/en/latest/) to narrow down our search critera and load the data.

In [5]:
# create a subcollection with certain search criteria
cat = col.search(
    variable_id=['tos', 'zos'],
    source_id=['CanESM5-CanOE', 'GFDL-ESM4'],
    experiment_id=['historical', 'ssp585'],
    grid_label='gn'
)

# load all datasets into a python dictionary
ddict = cat.to_dataset_dict(
    zarr_kwargs={'consolidated':True, 'use_cftime':True}, # recommended for faster reading and better time handling
    storage_options={'token': 'anon'}, # needed to access the public CMIP6 data on google
    aggregate=False,
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


The result is a python dictionary with many xarray Datasets

In [6]:
ds_a = ddict['CMIP.CCCma.CanESM5-CanOE.historical.r2i1p2f1.Omon.zos.gn.gs://cmip6/CMIP6/CMIP/CCCma/CanESM5-CanOE/historical/r2i1p2f1/Omon/zos/gn/v20190429/.nan.20190429']
ds_a

<xarray.Dataset>
Dimensions:             (i: 360, j: 291, time: 1980, bnds: 2, vertices: 4)
Coordinates:
  * i                   (i) int32 0 1 2 3 4 5 6 ... 353 354 355 356 357 358 359
  * j                   (j) int32 0 1 2 3 4 5 6 ... 284 285 286 287 288 289 290
    latitude            (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
  * time                (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:0...
    time_bnds           (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    zos                 (time, j, i) float32 dask.array<chunksize=(208, 291, 360), meta=np.ndarray>
Attributes: (12/58)
    CCCma_model_hash:            932b659de600c6a0e94f619abaf9cc79eabcd337
    CCCma_parent_runid:          canoecpl-007
    CCCma_pycmor_hash:           3ecdc18eb7c1f7fbce0346850f41adf815d9fb66
    CCCma_runid:                 c2-his02
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   1850:01:01:00
    ...                          ...
    variant_label:               r2i1p2f1
    version:                     v20190429
    netcdf_tracking_ids:         hdl:21.14100/02be791e-da03-4013-820c-2ad279c...
    version_id:                  v20190429
    intake_esm_varname:          None
    intake_esm_dataset_key:      CMIP.CCCma.CanESM5-CanOE.historical.r2i1p2f1...

In [7]:
ds_b = ddict['CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omon.zos.gn.gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/zos/gn/v20190726/.nan.20190726']
ds_b

<xarray.Dataset>
Dimensions:    (bnds: 2, y: 576, x: 720, vertex: 4, time: 1980)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    lat        (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * x          (x) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * y          (y) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Dimensions without coordinates: vertex
Data variables:
    zos        (time, y, x) float32 dask.array<chunksize=(61, 576, 720), meta=np.ndarray>
Attributes: (12/51)
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    0.0
    branch_time_in_parent:   36500.0
    comment:                 <null ref>
    ...                      ...
    variant_info:            N/A
    variant_label:           r1i1p1f1
    netcdf_tracking_ids:     hdl:21.14100/c11bbb98-a6ef-48de-81ee-e788ccdd134...
    version_id:              v20190726
    intake_esm_varname:      None
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omo...

Ok cool, that looks great, doesnt it? Why would I need cmip6_preprocessing? Well take a closer look at for instance the dimensions:

In [8]:
print(ds_a.dims)
print(ds_b.dims)

Frozen({'i': 360, 'j': 291, 'time': 1980, 'bnds': 2, 'vertices': 4})
Frozen({'bnds': 2, 'y': 576, 'x': 720, 'vertex': 4, 'time': 1980})


Oh damn, these are not the same and so if you want to average several model in space, you would have to write a ton of `if/else` statements...
This is the original reason I wrote cmip6_preprocessing: To take thes tedious bookkeeping tasks out and enable users to work with truly analysis ready data

In [9]:
from cmip6_preprocessing.preprocessing import combined_preprocessing
print(combined_preprocessing(ds_a).dims)
print(combined_preprocessing(ds_b).dims)

Frozen({'x': 360, 'y': 291, 'time': 1980, 'bnds': 2, 'vertex': 4})
Frozen({'x': 720, 'y': 576, 'vertex': 4, 'time': 1980, 'bnds': 2})


🚀 Now things look great. But for convenience you can also plug this functionality in your 'read-in' procedure:


In [10]:
# load all datasets into a python dictionary
ddict = cat.to_dataset_dict(
    zarr_kwargs={'consolidated':True, 'use_cftime':True}, # recommended for faster reading and better time handling
    storage_options={'token': 'anon'}, # needed to access the public CMIP6 data on google
    aggregate=False,
    preprocess=combined_preprocessing, # this applies the preprocessing to all datasets
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


In [1]:
# quick example of how to loop over several models
from cmip6_preprocessing.utils import cmip6_dataset_id
fig, axarr = plt.subplots(ncols=3, nrows=2, figsize=[8,4])
for ax, (name, ds) in zip(axarr.flat, ddict.items()):
    # select the first time step
    ds = ds.isel(time=0)
    # select the datavariable
    da = ds[ds.variable_id]
    # plot
    da.plot(ax=ax)
    ax.set_title('.'.join(cmip6_dataset_id(ds).split('.')[2:6]))
    
fig.subplots_adjust(hspace=0.3)

NameError: name 'plt' is not defined

In [ ]:
from cmip6_preprocessing.postprocessing import merge_variables, concat_members

In [ ]:
merge_variables(concat_members(ddict))

In [ ]:
concat_members(merge_variables(ddict))